In [1]:
import torch
from torch.utils.data import Dataset
from datasets import load_dataset


class OpenWebTextDataset(Dataset):
    def __init__(self, max_seq_len):
        # Load dataset
        dataset = load_dataset("stas/openwebtext-10k", split="train")

        # Combine all text into a single string
        self.text = " ".join(dataset["text"]).lower().split()
        
        # Create vocabulary
        vocab = sorted(set(self.text))
        self.vocab = {word: idx for idx, word in enumerate(vocab)}
        self.vocab_size = len(self.vocab)
        self.max_seq_len = max_seq_len

        # Convert text to token indices
        self.tokens = [self.vocab[word] for word in self.text]
        self.data = [
            self.tokens[i : i + max_seq_len] for i in range(len(self.tokens) - max_seq_len)
        ]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return torch.tensor(x), torch.tensor(y)

# Example usage
max_seq_len = 50
dataset = OpenWebTextDataset(max_seq_len)
print("Vocabulary size:", dataset.vocab_size)
print("Sample input:", dataset[0][0])
print("Sample target:", dataset[0][1])

c:\Users\Kadem\.pyenv\pyenv-win\versions\3.9.9\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
c:\Users\Kadem\.pyenv\pyenv-win\versions\3.9.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vocabulary size: 409917
Sample input: tensor([ 60731, 234155, 346236, 384786,  71337, 200785, 265126,  64556, 192909,
         71713, 354098, 358183,  15467, 369931,  94274, 209109, 281178, 202105,
         88883, 259525, 223774,  84455, 403675, 215483, 202105, 176896, 100388,
        354098, 180708, 265126,  86176, 193506, 354098, 122588,  71713, 183548,
        209912, 163456,  29267, 279409, 354098, 112154, 353913, 379347, 354098,
        106983, 265126,  64556, 192922])
Sample target: tensor([234155, 346236, 384786,  71337, 200785, 265126,  64556, 192909,  71713,
        354098, 358183,  15467, 369931,  94274, 209109, 281178, 202105,  88883,
        259525, 223774,  84455, 403675, 215483, 202105, 176896, 100388, 354098,
        180708, 265126,  86176, 193506, 354098, 122588,  71713, 183548, 209912,
        163456,  29267, 279409, 354098, 112154, 353913, 379347, 354098, 106983,
        265126,  64556, 192922, 151537])


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import math
import time
import os

In [3]:
class Pattention(nn.Module):
    """Pattention Layer.
    d1 = inputs dimension
    d2 = outpuuts dimension
    n = the number of parameters tokens representing the learnable keys and values
    """

    def __init__(
        self,
        d1,
        d2,
        n,
        param_key_init_method,
        param_value_init_method,
        norm_activation_type,
    ):
        super().__init__()

        self.param_token_num = n
        self.param_key_dim = d1
        self.param_value_dim = d2
        self.norm_activation_type = norm_activation_type
        
        self.key_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d1))) # Kp shape of (n, d1) 
        self.value_param_tokens = nn.parameter.Parameter(data=torch.rand((n, d2))) # Vp shape of (n, d2)
        
        param_key_init_method(self.key_param_tokens)
        param_value_init_method(self.value_param_tokens)
    
    def nonlinear_norm_func(self, inputs, normalize_type, dim=-1):
        if normalize_type == 'softmax': 
            # NOTE: softmax = exp_l1_norm
            # outputs = F.softmax(inputs, dim=dim) * inputs.shape[dim]
            nonlinear_outputs = torch.exp(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=1, dim=dim, keepdim=True) * inputs.shape[dim]
            outputs = norm_outputs
        elif normalize_type == 'gelu_l2_norm':
            nonlinear_outputs = F.gelu(inputs)
            norm_outputs = nonlinear_outputs / torch.norm(nonlinear_outputs, p=2, dim=dim, keepdim=True) * math.sqrt(nonlinear_outputs.shape[dim])
            outputs = norm_outputs
        elif normalize_type == 'l2_norm_gelu':
            norm_outputs = inputs / torch.norm(inputs, p=2, dim=dim, keepdim=True) * math.sqrt(inputs.shape[dim])
            nonlinear_outputs = F.gelu(norm_outputs)
            outputs = nonlinear_outputs
        return outputs

    def forward(self, inputs, dropout_p=0.0, attn_mask=None, scale=None):

        query = inputs
        key, value = self.key_param_tokens, self.value_param_tokens        
        L, S = query.size(-2), key.size(-2)
        scale_factor = 1 if scale is None else scale 
        # just for gelu nonlinear, set torch.zeros for softmax
        attn_bias = torch.ones(L, S, dtype=query.dtype, device=query.device)

        if attn_mask is not None:
            if attn_mask.dtype == torch.bool:
                # just for gelu nonlinear, set -inf for softmax
                attn_bias.masked_fill_(attn_mask.logical_not(), 0)
            else:
                raise NotImplementedError

        attn_weight = query @ key.transpose(-2, -1) * scale_factor
        # just for gelu nonlinear, set attn_weight += attn_bias for softmax
        attn_weight *= attn_bias
        # modified softmax
        attn_weight = self.nonlinear_norm_func(attn_weight, self.norm_activation_type, dim=-1)
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
        output = attn_weight @ value

        return output

In [4]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, attention_dropout=0.1,token_num=10):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads
        #print(hidden_size)
        #print(num_attention_heads)

        assert hidden_size % num_attention_heads == 0, "hidden_size must be divisible by num_attention_heads"

        # Query, Key, and Value projections
        self.query = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.key = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.value = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size // num_attention_heads,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )
        self.out_proj = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            #n=hidden_size,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )

        self.attention_dropout = nn.Dropout(attention_dropout)
        self.norm_factor = math.sqrt(self.head_dim)

    def forward(self, hidden_states, attention_mask=None):

        # Proceed with attention mechanism
        batch_size, seq_len, _ = hidden_states.size()

        query_layer = self.query(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("qshape: ", query_layer.shape)
        key_layer = self.key(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("kshape: ", key_layer.shape)
        value_layer = self.value(hidden_states).view(
            batch_size, seq_len, self.num_attention_heads, self.head_dim
        )
        #print("vshape: ", value_layer.shape)
        query_layer = query_layer.transpose(1, 2)
        key_layer = key_layer.transpose(1, 2)
        value_layer = value_layer.transpose(1, 2)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores /= self.norm_factor

        if attention_mask is not None:
            attention_scores += attention_mask

        attention_probs = torch.softmax(attention_scores, dim=-1)
        attention_probs = self.attention_dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.transpose(1, 2).contiguous().view(
            batch_size, seq_len, self.hidden_size
        )

        output = self.out_proj(context_layer)
        return output

In [5]:
class TokenformerLayer(nn.Module):
    """A single Tokenformer layer implementing token-token and token-parameter interactions."""

    def __init__(
        self,
        hidden_size,
        vocab_size,
        num_attention_heads,
        max_seq_len,
        attention_dropout=0.1,
        hidden_dropout=0.1,
        token_num=10
    ):
        """
        Args:
            hidden_size (int): The size of the hidden dimension.
            num_attention_heads (int): Number of attention heads for multi-head attention.
            num_param_tokens (int): Number of parameter tokens for the feed-forward Pattention layer.
            attention_dropout (float): Dropout probability for attention weights.
            hidden_dropout (float): Dropout probability for residual connections.
        """
        super().__init__()
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads

        assert (
            hidden_size % num_attention_heads == 0
        ), "hidden_size must be divisible by num_attention_heads"

        # Layer normalizations
        self.input_layernorm = nn.LayerNorm(hidden_size)
        self.post_attention_layernorm = nn.LayerNorm(hidden_size)

        # Token and positional embeddings
        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_seq_len, hidden_size)

        # Self-attention using Pattention
        self.attention = self.attention = SelfAttention(
            #vocab_size=30522,  # Provide a valid vocab_size if needed
            hidden_size=hidden_size,
            num_attention_heads=num_attention_heads,
            #max_seq_len=max_seq_len,
            attention_dropout=attention_dropout,
            token_num=token_num
        )

        # Feed-forward network using Pattention
        self.mlp = Pattention(
            d1=hidden_size,
            d2=hidden_size,
            n=token_num,
            param_key_init_method=torch.nn.init.xavier_uniform_,
            param_value_init_method=torch.nn.init.xavier_uniform_,
            norm_activation_type="l2_norm_gelu"
        )

        self.hidden_dropout = hidden_dropout
        self.dropout = nn.Dropout(hidden_dropout)

        # to obtain logits (before softmax) for the vocabulary
        self.lm_head = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, attention_mask=None):
        """
        Forward pass for the Tokenformer layer.
        
        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, seq_len, hidden_size].
            attention_mask (torch.Tensor, optional): Attention mask for self-attention.

        Returns:
            torch.Tensor: Output tensor of the same shape as the input.
        """
        #print(f"Input shape before LayerNorm: {x.shape}")
        # Residual connection and pre-normalization for attention
        # Word embedding
        x = self.token_embedding(x)  # [batch_size, seq_len, hidden_size]
        #print("token emb: ", x.shape)
        # Positional embedding
        seq_len = x.size(1)
        position_ids = torch.arange(seq_len, device=x.device).unsqueeze(0)  # [1, seq_len]
        #print("Position: ", position_ids.shape)
        x += self.position_embedding(position_ids)  # Add positional embeddings
        #print( "emb + pos", x.shape)

        residual = x
        normed_input = self.input_layernorm(x)
        #print("norme: ", normed_input.shape)

        # Self-attention
        attention_output = self.attention(normed_input, attention_mask)
        #print("attention: ", attention_output.shape)
        attention_output = self.dropout(attention_output) + residual

        # Residual connection and pre-normalization for feed-forward
        residual = attention_output
        normed_attention_output = self.post_attention_layernorm(attention_output)

        # Feed-forward network (Pattention)
        mlp_output = self.mlp(normed_attention_output)
        #print("feed: ",  mlp_output.shape)
        output = self.dropout(mlp_output) + residual

        # Linear layer for logits
        logits = self.lm_head(output)

        return logits

In [ ]:
def estimate_perplexity(model, dataloader, criterion, device):
    """
    Compute the perplexity of a language model.
    """
    model.eval()  # Set the model to evaluation mode.
    total_loss = 0
    total_tokens = 0

    # Disable gradient computation for evaluation.
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            # Move the batch data to the specified device.
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass through the model.
            outputs = model(x_batch)  # Shape: (batch_size, seq_len, vocab_size)
            outputs = outputs.view(-1, outputs.shape[-1])  # Reshape to (batch_size * seq_len, vocab_size)
            y_batch = y_batch.view(-1)  # Flatten targets to shape: (batch_size * seq_len)

            # Compute the loss for the current batch.
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)  # Multiply by the number of tokens.
            total_tokens += y_batch.size(0)

    # Calculate the average loss per token.
    avg_loss = total_loss / total_tokens
    # Compute the perplexity by exponentiating the average loss.
    perplexity = torch.exp(torch.tensor(avg_loss))
    return avg_loss, perplexity.item()


from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

def train(use_metrics=False):
    # Configurations
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")  # use M1 GPU
    else:
        device = torch.device("cpu")  # use CPU if no M1 GPU
    print('Device:', device)

    hidden_dim = 32
    num_heads = 4
    max_seq_len = 16  # Réduction de la taille des séquences
    batch_size = 8  # Batch réduit
    num_epochs = 5  # Moins d'époques pour tester plus vite
    learning_rate = 0.001
    token_num = 32

    # Préparer les données
    dataset = OpenWebTextDataset(max_seq_len=max_seq_len)

    # Réduction du dataset
    subset_size = 100_000  # Choisir un sous-ensemble plus petit
    dataset = Subset(dataset, range(min(subset_size, len(dataset))))

    vocab_size = dataset.dataset.vocab_size  # Mise à jour du vocab_size

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print("Dataset réduit à :", len(dataset))
    print("Vocab size:", vocab_size)

    # Instancier le modèle
    model = TokenformerLayer(hidden_dim, vocab_size, num_heads, max_seq_len, token_num=token_num)
    model = model.to(device)
    print("Model parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

    # Définir la fonction de perte et l'optimiseur
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print("Début de l'entraînement")
    total_training_time = 0.0
    overall_start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        epoch_train_start = time.time()

        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for x_batch, y_batch in progress_bar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            outputs = model(x_batch)
            outputs = outputs.view(-1, vocab_size)
            y_batch = y_batch.view(-1)

            loss = criterion(outputs, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

        epoch_training_time = time.time() - epoch_train_start
        total_training_time += epoch_training_time

        if use_metrics:
            eval_start = time.time()
            avg_loss, perplexity = estimate_perplexity(model, dataloader, criterion, device)
            eval_time = time.time() - eval_start
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {epoch_loss / len(dataloader):.4f}, "
                  f"Eval Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}, "
                  f"Train Time: {epoch_training_time:.2f}s, Eval Time: {eval_time:.2f}s")
        else:
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {epoch_loss / len(dataloader):.4f}, "
                  f"Train Time: {epoch_training_time:.2f}s")

    print(f"Total training time (excluding evaluation): {total_training_time:.2f} seconds")
    # Sauvegarde du modèle après entraînement
    model_save_path = "tokenformer_model.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Modèle sauvegardé dans : {model_save_path}")
    return model


In [7]:
hidden_dim = 32
num_heads = 4
max_seq_len = 16  # Réduction de la taille des séquences
batch_size = 8  # Batch réduit
num_epochs = 3  # Moins d'époques pour tester plus vite
learning_rate = 0.001
token_num = 32



In [8]:
# Préparer les données
dataset = OpenWebTextDataset(max_seq_len=max_seq_len)

print("Nombre total d'échantillons :", len(dataset))


Nombre total d'échantillons : 8193712


In [17]:
train(use_metrics=True)

Device: cuda
Dataset réduit à : 100000
Vocab size: 409917
Model parameters: 26655485
Début de l'entraînement


Epoch 1/5, Train Loss: 3.2687, Eval Loss: 0.8543, Perplexity: 2.3497, Train Time: 244.47s, Eval Time: 66.78s


Epoch 2/5, Train Loss: 0.8479, Eval Loss: 0.5063, Perplexity: 1.6592, Train Time: 235.62s, Eval Time: 63.27s


Epoch 3/5, Train Loss: 0.6201, Eval Loss: 0.4513, Perplexity: 1.5703, Train Time: 253.95s, Eval Time: 60.19s


Epoch 4/5, Train Loss: 0.5538, Eval Loss: 0.4309, Perplexity: 1.5387, Train Time: 247.08s, Eval Time: 63.85s


Epoch 5/5, Train Loss: 0.5202, Eval Loss: 0.4167, Perplexity: 1.5169, Train Time: 238.87s, Eval Time: 68.79s
Total training time (excluding evaluation): 1220.00 seconds
